In [1]:
import pandas as pd
import re
from pathlib import Path
import pyarabic.araby as ar
from tqdm.notebook import tqdm

## Config

In [2]:
class Config:
    raw_data_file = 'tweets.csv'
    processed_data_file = 'tweets.csv'
    
    RAW_DATA_PATH = Path('../../data/raw')
    PROCESSED_DATA_PATH = Path('../../data/processed')
    
    RAW_DATA_FILE = RAW_DATA_PATH / raw_data_file
    PROCESSED_DATA_FILE = PROCESSED_DATA_PATH / processed_data_file

## Constants

In [3]:
# match decoration inside words
in_word_re = re.compile(r"[\u0600-\u0603\u0610-\u061a\u0640\u064b-\u065f\u06fa-\u06ff\u06ea-\u06ef\u06df-\u06e8\u06d4"
                        r"\u06d5\u06bf-\u06cb]")

ARABIC_LETTERS = [
    'ء', 'آ', 'أ', 'ؤ', 'إ', 'ئ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س',
    'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ى', 'ي', ' '
]

NON_ARABIC_LETTER = re.compile('[^' + ''.join(ARABIC_LETTERS + [' ']) + ']')


## Read Data

In [4]:
raw_data = pd.read_csv(str(Config.RAW_DATA_FILE), sep=',', names=['ID','Tweet','Class'])

In [5]:
raw_data.head()

,ID,Tweet,Class
0,295820374774513667,"رودجرز"" انا مستاء من لاعبينا الشباب. حولنا من...",anger
1,353619287652581376,‏من شفت انا عيونه اعلنت انا جنوني ♥,anger
2,888346524714979329,أتفق انا عصِبي وقاسي وعنيد ومُتناقض ومُزاجي وك...,anger
3,870974517983752193,@AWADBADINAHAS على فكرة انا مستاء جداً من تجرب...,anger
4,44022,Power outage im scared,anger


## Process Data

In [6]:
class DataPreprocessor:

    @staticmethod
    def process_text(text):
        # trim text
        text = text.strip()
        # strip decoration inside words
        text = in_word_re.sub('', text)
        # strip shadda
        text = ar.strip_shadda(text)
        # strip tashkeel
        text = ar.strip_tashkeel(text)
        # strip tatweel
        text = ar.strip_tatweel(text)
        # strip non arabic letter
        text = re.sub(NON_ARABIC_LETTER, ' ', text)
        # strip multiple spaces with single one
        text = ' '.join(text.split())
        return text


In [7]:
processed_data = raw_data.copy()

**Drop NaN rows**

In [8]:
print(f'size before drop nan rows: {len(processed_data)}')
processed_data = processed_data.dropna().reset_index(drop=True)
print(f'size after drop nan rows: {len(processed_data)}')

size before drop nan rows: 94832
size after drop nan rows: 94827


**Clean Tweets**

In [9]:
processed_data['Tweet'] = processed_data['Tweet'].apply(lambda tweet: DataPreprocessor.process_text(tweet))

**Drop Empty Tweets**

In [10]:
print(f'size before drop empty rows: {len(processed_data)}')
indexes = processed_data[processed_data['Tweet'].apply(lambda tweet: len(str(tweet).strip()) == 0)].index
processed_data = processed_data.drop(indexes).reset_index(drop=True)
print(f'size after drop empty rows: {len(processed_data)}')

size before drop empty rows: 94827
size after drop empty rows: 80668


In [11]:
processed_data.head()

,ID,Tweet,Class
0,295820374774513667,رودجرز انا مستاء من لاعبينا الشباب حولنا منحهم...,anger
1,353619287652581376,من شفت انا عيونه اعلنت انا جنوني,anger
2,888346524714979329,أتفق انا عصبي وقاسي وعنيد ومتناقض ومزاجي وكتوم...,anger
3,870974517983752193,على فكرة انا مستاء جدا من تجربتي في محلكم في ا...,anger
4,886636321342619653,ع قد ما انا عصبي ع قد ما ابسط كلمه حلوه هتنسين...,anger


In [12]:
processed_data['Class'].value_counts()

happiness       22788
trust           13403
sadness         12906
anticipation    11785
fear            11182
surprise         6679
anger            1154
disgust           771
Name: Class, dtype: int64

In [13]:
processed_data.to_csv(Config.PROCESSED_DATA_FILE, index=False)